In [ ]:
!aws s3 ls s3://airetail/retail_db_json/daily_product_revenue/

In [ ]:
import boto3

In [ ]:
rsd_client = boto3.client('redshift-data')

In [ ]:
table_name = 'daily_product_revenue'

In [ ]:
secret_arn = '<REPLACE WITH YOUR SECRET ARN>'

In [ ]:
truncate_table_cmd = f'''TRUNCATE TABLE public.{table_name}'''

In [ ]:
truncate_table_cmd_id = rsd_client.execute_statement(
    WorkgroupName='retail-wg',
    Database='retail_dm',
    SecretArn=secret_arn,
    Sql=truncate_table_cmd
)['Id']

In [ ]:
rsd_client.describe_statement(Id=truncate_table_cmd_id)

In [ ]:
count_query = f'''SELECT count(*) FROM {table_name}'''

In [ ]:
count_query_id = rsd_client.execute_statement(
    WorkgroupName='retail-wg',
    Database='retail_dm',
    SecretArn=secret_arn,
    Sql=count_query
)['Id']

In [ ]:
rsd_client.describe_statement(Id=count_query_id)

In [ ]:
rsd_client.get_statement_result(Id=count_query_id)['Records']

In [ ]:
bucket_name = 'airetail'


In [ ]:
rs_copy_command = f'''
    COPY {table_name} FROM 's3://{bucket_name}/retail_db_json/{table_name}' 
    IAM_ROLE 'arn:aws:iam::269066542444:role/AIRedshiftRetailSpectrum'
    JSON AS 'auto'
'''

In [ ]:
from botocore.exceptions import WaiterError
from botocore.waiter import WaiterModel
from botocore.waiter import create_waiter_with_client

In [ ]:
waiter_name = 'DataAPIExecution'

delay=2
max_attempts=3

#Configure the waiter settings
waiter_config = {
  'version': 2,
  'waiters': {
    'DataAPIExecution': {
      'operation': 'DescribeStatement',
      'delay': delay,
      'maxAttempts': max_attempts,
      'acceptors': [
        {
          "matcher": "path",
          "expected": "FINISHED",
          "argument": "Status",
          "state": "success"
        },
        {
          "matcher": "pathAny",
          "expected": ["PICKED","STARTED","SUBMITTED"],
          "argument": "Status",
          "state": "retry"
        },
        {
          "matcher": "pathAny",
          "expected": ["FAILED","ABORTED"],
          "argument": "Status",
          "state": "failure"
        }
      ],
    },
  },
}

In [ ]:
waiter_model = WaiterModel(waiter_config)

In [ ]:
custom_waiter = create_waiter_with_client(waiter_name, waiter_model, rsd_client)

In [ ]:
rs_copy_command_id = rsd_client.execute_statement(
    WorkgroupName='retail-wg',
    Database='retail_dm',
    SecretArn=secret_arn,
    Sql=rs_copy_command
)['Id']

try:
    custom_waiter.wait(Id=rs_copy_command_id)    
except WaiterError as e:
    print (e)


In [ ]:
rsd_client.describe_statement(Id=rs_copy_command_id)

In [ ]:
count_query = f'''SELECT count(*) FROM public.{table_name}'''

In [ ]:
count_query_id = rsd_client.execute_statement(
    WorkgroupName='retail-wg',
    Database='retail_dm',
    SecretArn=secret_arn,
    Sql=count_query
)['Id']

In [ ]:
rsd_client.describe_statement(Id=count_query_id)

In [ ]:
rsd_client.get_statement_result(Id=count_query_id)['Records']